In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
import folium
import folium.plugins as plugins
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
import geopy.distance
import math
from pandas import option_context
import json
from collections import Counter

In [13]:
data = pd.read_csv("./data/pickup_delivery_occurences_every_hour.csv", index_col=0)

In [14]:
data.head()

,Zone,Period,Pickup occurences,Delivery occurences
0,1,0,"{""0"": 362, ""1"": 3}","{""0"": 362, ""1"": 3}"
1,1,1,"{""0"": 362, ""1"": 3}","{""0"": 360, ""1"": 4, ""2"": 1}"
2,1,2,"{""0"": 363, ""1"": 2}","{""0"": 361, ""1"": 3, ""2"": 1}"
3,1,3,"{""0"": 364, ""1"": 1}","{""0"": 364, ""1"": 1}"
4,1,4,"{""0"": 365}","{""0"": 364, ""1"": 1}"


In [15]:
with option_context('display.max_colwidth', 400):
    display(data.loc[(data['Zone'] == 126)])

,Zone,Period,Pickup occurences,Delivery occurences
3000,126,0,"{""0"": 329, ""1"": 26, ""2"": 9, ""3"": 1}","{""0"": 315, ""1"": 44, ""2"": 6}"
3001,126,1,"{""0"": 334, ""1"": 23, ""2"": 6, ""3"": 2}","{""0"": 333, ""1"": 29, ""2"": 2, ""3"": 1}"
3002,126,2,"{""0"": 333, ""1"": 27, ""2"": 4, ""3"": 1}","{""0"": 344, ""1"": 15, ""2"": 6}"
3003,126,3,"{""0"": 338, ""1"": 22, ""2"": 5}","{""0"": 354, ""1"": 9, ""2"": 2}"
3004,126,4,"{""0"": 348, ""1"": 13, ""2"": 4}","{""0"": 358, ""1"": 7}"
3005,126,5,"{""0"": 310, ""1"": 48, ""2"": 6, ""3"": 1}","{""0"": 355, ""1"": 10}"
3006,126,6,"{""0"": 229, ""1"": 102, ""2"": 24, ""3"": 7, ""4"": 3}","{""0"": 320, ""1"": 42, ""2"": 3}"
3007,126,7,"{""0"": 193, ""1"": 101, ""2"": 50, ""3"": 17, ""4"": 4}","{""0"": 289, ""1"": 67, ""2"": 9}"
3008,126,8,"{""0"": 224, ""1"": 92, ""2"": 41, ""3"": 5, ""4"": 3}","{""0"": 292, ""1"": 64, ""2"": 8, ""4"": 1}"
3009,126,9,"{""0"": 266, ""1"": 82, ""2"": 13, ""3"": 4}","{""0"": 276, ""1"": 79, ""2"": 9, ""3"": 1}"


In [318]:
data['Pickup distribution'] = np.nan
data['Max value deliveries'] = np.nan
data['Pickup distribution length match'] = np.nan
len(data)

6096

In [319]:
data = pd.read_csv("./data/pickup_delivery_occurences_every_hour.csv", index_col=0)

data['Pickup distribution'] = np.nan
data['Delivery distribution'] = np.nan

# pickups
for i in range(0,6096):
    
    d = json.loads(data.iloc[i,2])
    keys = list(range(int(max(d, key=int))+1))
    change = False
    for key in keys:
        if (str(key) not in d):
            change = True
            d[str(key)] = 0
    json_d = json.dumps(d)
    if change:
        data.iloc[i,2] = str(json_d)
        
        out_list = [round(d[str(k)]/365,3) for k in keys]
        data.iloc[i,4] = str(out_list)
    
    else:
        out_list = [round(d[str(k)]/365, 3) for k in keys]
        data.iloc[i,4] = str(out_list)

        
# deliveries
for i in range(0,762):
    
    d = json.loads(data.iloc[i,3])
    keys = list(range(int(max(d, key=int))+1))
    change = False
    for key in keys:
        if (str(key) not in d):
            change = True
            d[str(key)] = 0
    json_d = json.dumps(d)
    if change:
        data.iloc[i,3] = str(json_d)
        
        out_list = [round(d[str(k)]/365,3) for k in keys]
        data.iloc[i,5] = str(out_list)
    # Fill pickup distribution
    else:
        out_list = [round(d[str(k)]/365, 3) for k in keys]
        data.iloc[i,5] = str(out_list)
    
    

In [18]:
data = pd.read_csv("./data/pickup_delivery_distributions_ever_hour.csv", index_col=0)

In [39]:
with option_context('display.max_colwidth', 400):
    display(data.head(764))


,Zone,Period,Pickup occurences,Delivery occurences,Pickup distribution,Delivery distribution
0,1,0,"{""0"": 362, ""1"": 3}","{""0"": 362, ""1"": 3}","[0.992, 0.008]","[0.992, 0.008]"
1,1,1,"{""0"": 362, ""1"": 3}","{""0"": 360, ""1"": 4, ""2"": 1}","[0.992, 0.008]","[0.986, 0.011, 0.003]"
2,1,2,"{""0"": 363, ""1"": 2}","{""0"": 361, ""1"": 3, ""2"": 1}","[0.995, 0.005]","[0.989, 0.008, 0.003]"
3,1,3,"{""0"": 364, ""1"": 1}","{""0"": 364, ""1"": 1}","[0.997, 0.003]","[0.997, 0.003]"
4,1,4,"{""0"": 365}","{""0"": 364, ""1"": 1}",[1.0],"[0.997, 0.003]"
...,...,...,...,...,...,...
759,32,15,"{""0"": 322, ""1"": 41, ""2"": 2}","{""0"": 326, ""1"": 36, ""2"": 3}","[0.882, 0.112, 0.005]","[0.893, 0.099, 0.008]"
760,32,16,"{""0"": 316, ""1"": 47, ""2"": 1, ""3"": 1}","{""0"": 302, ""1"": 55, ""2"": 8}","[0.866, 0.129, 0.003, 0.003]","[0.827, 0.151, 0.022]"
761,32,17,"{""0"": 308, ""1"": 52, ""2"": 4, ""3"": 1}","{""0"": 294, ""1"": 60, ""2"": 9, ""3"": 2}","[0.844, 0.142, 0.011, 0.003]","[0.805, 0.164, 0.025, 0.005]"
762,32,18,"{""0"": 305, ""1"": 50, ""2"": 8, ""3"": 2}","{""0"": 297, ""1"": 56, ""2"": 9, ""3"": 3}","[0.836, 0.137, 0.022, 0.005]",NaN


In [325]:
#data.to_csv("./data/pickup_delivery_distributions_ever_hour.csv")

## Create dataset with zone coordinates

In [9]:
zone_data = pd.read_csv("./data/2019_vy_cleaned_prepared.csv")

In [10]:
zone_data.head()

,DateTime,Trip ID,TripStage,Long,Lat,Car ID,Trip duration (min),Zone,No. trips zone
0,2018-12-31 11:39:21.888000000,1,Start,10.73382,59.92934,V5a15A3226E5F7,1403.350000,109.0,2482.0
1,2019-01-01 11:02:43.008000000,1,End,10.74900,59.91063,V5a15A3226E5F7,1403.350000,189.0,6666.0
2,2019-01-01 08:18:08.352000000,2,Start,10.73561,59.93056,V5a15A3226E89D,24.716667,109.0,2482.0
3,2019-01-01 08:42:50.976000000,2,End,10.80917,59.94132,V5a15A3226E89D,24.716667,64.0,999.0
4,2019-01-01 14:34:39.360000000,3,Start,10.72983,59.92513,V5a15A3226E999,1067.866667,129.0,3072.0


In [ ]:
a